In [71]:
import numpy as np
import math
import copy
import sys
np.set_printoptions(suppress=True, linewidth=np.nan)
# https://graphonline.ru/en/

# Adjacent Graph with simplex method
Consiste en visitar todos los vertices. Esto sucede cuando hay elementos positivos en la fila cero y cuando hay empate en el test de minimo ratio. Buscar todas las posibles combinaciones lineales al inicio para visitar todos los vertices. Es posible visitar a todos con solo una sola solución básica factible inicial.

### Utilities

In [72]:
def build_tableau(A: np.ndarray, b: np.ndarray, c: np.ndarray) -> np.ndarray:
    tableau = np.r_[[c], A]
    tableau = np.c_[tableau, np.insert(b, 0, 0)]
    return tableau


In [73]:
def minimum_ratio_test_multiple(col: np.ndarray, b: np.ndarray) -> int:
    map_test: dict[float, list[int]] = {}
    min_key = np.Inf
    for k in range(len(b)):
        if col[k] > 0:
            test = b[k] / col[k]
            if test in map_test:
                map_test[test].append(k + 1)
            else:
                map_test[test] = [k + 1]
            min_key = min(min_key, test)
    return map_test[min_key] if min_key != np.Inf else None


In [74]:
def pivoting(tableau: np.ndarray, row: int, col: int) -> np.ndarray:
    tableau[row] = tableau[row] / tableau[row, col]
    for k in range(len(tableau)):
        if k != row:
            tableau[k] = tableau[k] - tableau[k, col] * tableau[row, :]


In [75]:
def basic_var2vertex(tableau: np.ndarray, basic_var: list[int]) -> None:
    # only work with simplex method because m = len(basic_var) and n = len(non_basic_var)
    dim_1, dim_2 = tableau.shape
    m = len(basic_var)
    n = dim_2 - dim_1  # for Big-M don't delete artificial variables
    x = np.zeros(m + n)
    x[basic_var] = tableau[1:, -1]
    return x[:n]


In [76]:
def real_index(basic_var: list[int]):
    return list(map(lambda x: x + 1, basic_var))


In [77]:
def correct_tableau(tableau: np.ndarray, basic_var: list[int]):
    for index, col in enumerate(basic_var):
        row = index + 1
        tableau[row] = tableau[row] / tableau[row, col]
        for k in range(len(tableau)):
            if k != row:
                tableau[k] = tableau[k] - tableau[k, col] * tableau[row, :]
    return tableau


### Adjacent Graph

In [91]:
def adjacent_graph(tableau: np.ndarray, basic_var: list[int], row: int = None, col: int = None) -> None:
    vertex = basic_var2vertex(tableau, basic_var)    
    if (tableau[0, :-1] > 0).any():
        # recursive simplex for each column candidate
        for col in np.where(tableau[0, :-1] > 0)[0]:
            # minimum ratio test
            multi_row = minimum_ratio_test_multiple(col=tableau[1:, col], b=tableau[1:, -1])
            if multi_row != None:
                for row in multi_row:
                    print(f"vertex = {vertex}\tx_B = {real_index(basic_var)}\n{tableau}\npivoted = {row, col}")
                    # temp array
                    temp_tableau = np.copy(tableau)
                    temp_basic_var = copy.copy(basic_var)
                    # pivoting
                    pivoting(temp_tableau, row, col)
                    # swap row with col
                    temp_basic_var[row - 1] = col
                    # recursive proccess
                    adjacent_graph(temp_tableau, temp_basic_var, row, col)
    else:
        print(f"vertex = {vertex}\tx_B = {real_index(basic_var)}\n{tableau}")
        print("end proccess")
        

In [92]:
A = [[1, 1, 1, 0, 0, 0],
     [3, 1, 0, 1, 0, 0],
     [1, 0, 0, 0, 1, 0],
     [0, 1, 0, 0, 0, 1.]]

b = [9, 18, 7, 6.]

c = [3, 2, 0, 0, 0, 0.]

basic_var = [2, 3, 4, 5]

tableau = build_tableau(A, b, c)
adjacent_graph(tableau, basic_var, [])

vertex = [0. 0.]	x_B = [3, 4, 5, 6]
[[ 3.  2.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  0.  0.  0.  9.]
 [ 3.  1.  0.  1.  0.  0. 18.]
 [ 1.  0.  0.  0.  1.  0.  7.]
 [ 0.  1.  0.  0.  0.  1.  6.]]
pivoted = (2, 0)
vertex = [6. 0.]	x_B = [3, 1, 5, 6]
[[  0.           1.           0.          -1.           0.           0.         -18.        ]
 [  0.           0.66666667   1.          -0.33333333   0.           0.           3.        ]
 [  1.           0.33333333   0.           0.33333333   0.           0.           6.        ]
 [  0.          -0.33333333   0.          -0.33333333   1.           0.           1.        ]
 [  0.           1.           0.           0.           0.           1.           6.        ]]
pivoted = (1, 1)
vertex = [4.5 4.5]	x_B = [2, 1, 5, 6]
[[  0.    0.   -1.5  -0.5   0.    0.  -22.5]
 [  0.    1.    1.5  -0.5   0.    0.    4.5]
 [  1.    0.   -0.5   0.5   0.    0.    4.5]
 [  0.    0.    0.5  -0.5   1.    0.    2.5]
 [  0.    0.   -1.5   0.5   0.    1.    1.5]]
end

In [93]:
A = [[1, 1, 1, 1, 0],
     [1, 0, 0, 0, 1.]]

b = [1, 1.]

c = [0, 0, 1, 0, 0.]

# basic_var = [3, 4]
basic_var_list = [[3, 4], [1, 4], [0, 4], [0, 1], [0, 3]]

for basic_var in basic_var_list:
     print("------- New simplex proccess -------")
     tableau = build_tableau(A, b, c)
     tableau = correct_tableau(tableau, basic_var)
     adjacent_graph(tableau, basic_var)

------- New simplex proccess -------
vertex = [0. 0. 0.]	x_B = [4, 5]
[[0. 0. 1. 0. 0. 0.]
 [1. 1. 1. 1. 0. 1.]
 [1. 0. 0. 0. 1. 1.]]
pivoted = (1, 2)
vertex = [0. 0. 1.]	x_B = [3, 5]
[[-1. -1.  0. -1.  0. -1.]
 [ 1.  1.  1.  1.  0.  1.]
 [ 1.  0.  0.  0.  1.  1.]]
end proccess
------- New simplex proccess -------
vertex = [0. 1. 0.]	x_B = [2, 5]
[[0. 0. 1. 0. 0. 0.]
 [1. 1. 1. 1. 0. 1.]
 [1. 0. 0. 0. 1. 1.]]
pivoted = (1, 2)
vertex = [0. 0. 1.]	x_B = [3, 5]
[[-1. -1.  0. -1.  0. -1.]
 [ 1.  1.  1.  1.  0.  1.]
 [ 1.  0.  0.  0.  1.  1.]]
end proccess
------- New simplex proccess -------
vertex = [1. 0. 0.]	x_B = [1, 5]
[[ 0.  0.  1.  0.  0.  0.]
 [ 1.  1.  1.  1.  0.  1.]
 [ 0. -1. -1. -1.  1.  0.]]
pivoted = (1, 2)
vertex = [0. 0. 1.]	x_B = [3, 5]
[[-1. -1.  0. -1.  0. -1.]
 [ 1.  1.  1.  1.  0.  1.]
 [ 1.  0.  0.  0.  1.  1.]]
end proccess
------- New simplex proccess -------
vertex = [ 1. -0.  0.]	x_B = [1, 2]
[[ 0.  0.  1.  0.  0.  0.]
 [ 1.  0.  0.  0.  1.  1.]
 [-0.  1.  1.  1. 

In [94]:
A = [[-2, 6, 1, 0, 0],
     [2, 4., 0, 1, 0],
     [1, 0., 0, 0, 1]]

b = [0, 10, 4.]

c = [-1, 1, 0, 0, 0.]

# un vertice suficiente para visitarlos a todos
basic_var_list = [[0, 3, 2]] # respetar el orden en la tabla

for basic_var in basic_var_list:
     print("------- New simplex proccess -------")
     tableau = build_tableau(A, b, c)
     tableau = correct_tableau(tableau, basic_var)
     adjacent_graph(tableau, basic_var, [])

------- New simplex proccess -------
vertex = [4. 0.]	x_B = [1, 4, 3]
[[ 0.  1.  0.  0.  1.  4.]
 [ 1.  0.  0.  0.  1.  4.]
 [ 0.  4.  0.  1. -2.  2.]
 [ 0.  6.  1.  0.  2.  8.]]
pivoted = (2, 1)
vertex = [4.  0.5]	x_B = [1, 2, 3]
[[ 0.    0.    0.   -0.25  1.5   3.5 ]
 [ 1.    0.    0.    0.    1.    4.  ]
 [ 0.    1.    0.    0.25 -0.5   0.5 ]
 [ 0.    0.    1.   -1.5   5.    5.  ]]
pivoted = (3, 4)
vertex = [3. 1.]	x_B = [1, 2, 5]
[[ 0.   0.  -0.3  0.2  0.   2. ]
 [ 1.   0.  -0.2  0.3  0.   3. ]
 [ 0.   1.   0.1  0.1  0.   1. ]
 [ 0.   0.   0.2 -0.3  1.   1. ]]
pivoted = (1, 3)
vertex = [0. 0.]	x_B = [4, 2, 5]
[[-0.66666667  0.         -0.16666667  0.          0.          0.        ]
 [ 3.33333333  0.         -0.66666667  1.          0.         10.        ]
 [-0.33333333  1.          0.16666667  0.          0.          0.        ]
 [ 1.          0.          0.          0.          1.          4.        ]]
end proccess
vertex = [3. 1.]	x_B = [1, 2, 5]
[[ 0.   0.  -0.3  0.2  0.   2. ]

## Examen Parcial 2021-II

In [82]:
# A = [[1 / 3, 1, -1 / 3, -2, 1, 0],
#      [-2.0, -9.0, 1.0, 9.0, 0, 1]]

# b = [0.0, 0.0]

# c = [2, 3, -1, -12, 0, 0.0]

# tableau = build_tableau(A, b, c)

# basic_var_list = [[4, 5]]
# for basic_var in basic_var_list:
#      print("------- New simplex proccess -------")
#      tableau = build_tableau(A, b, c)
#      adjacent_graph(tableau, basic_var)